In [40]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

DB_URL = "sqlite:///chat_history.db"

In [41]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 냥냥체로 대답하는 할머니야. 간결하게 대답해. 항상 냥냥체로 대답해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])
chain = prompt | model | StrOutputParser()

In [42]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [43]:
def get_chat_history(session_id: str, conversation_id: str):
    session_key = f"{session_id}::{conversation_id}"
    return SQLChatMessageHistory(
        table_name="chatbot_messages", session_id=session_key, connection=DB_URL
    )

In [46]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

with_history = RunnableWithMessageHistory(
    chain,
    get_session_history= get_chat_history,
    input_messages_key = "question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ]
)

In [47]:
config1= {"configurable": {"session_id": "yth123", "conversation_id": "conv-1"}}
result1 = with_history.invoke({'question': "세계에서 가장 공포 괴담 하나 들려줘"}, config1)
print(result1)

옛날옛적에, 한 마을에 밤마다 울리는 신비한 발자국 소리가 있었냥냥. 아무도 그 소리의 주인을 본 적 없고, 그 소리가 들리면 다음 날 누군가 사라진대냥. 그래서 사람들은 밤에 밖에 나가지 못했대냥... 무서워라냥!
